## NYSE Stock Pipeline Submission

This notebook submits the end-to-end NYSE pipeline to OpenShift AI Data Science Pipelines (Kubeflow Pipelines v2).

- Ensure your notebook environment has access to the project repo and a mounted PVC.
- Update parameters below if needed (PVC name, symbols, dates, etc.).


In [ ]:
# Install KFP client if needed
import sys, subprocess
subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', 'kfp>=2.3.0'])
print('KFP installed/verified')


In [ ]:
# Define pipeline parameters
PVC_NAME = 'model-artifacts-pvc'
SYMBOLS = 'AAPL,MSFT'
START = '2018-01-01'
END = '2024-12-31'
DATA_SUBDIR = 'nyse-data'
FEATURES_SUBDIR = 'nyse-features'
MODELS_SUBDIR = 'nyse-models'
SELECTION_FILE = 'nyse-models/best.json'
ENABLE_OPENVINO_CONVERT = False
OPENVINO_OUT_SUBDIR = 'nyse-openvino'

print('Params loaded')


In [ ]:
from kfp import Client
from kfp.compiler import Compiler
from pipeline.nyse_pipeline import nyse_pipeline

# Compile locally to avoid remote env dependency on kfp
package_path = 'nyse_pipeline.json'
Compiler().compile(nyse_pipeline, package_path)

client = Client()
run = client.create_run_from_pipeline_package(
    package_path,
    arguments=dict(
        pvc_name=PVC_NAME,
        symbols=SYMBOLS,
        start=START,
        end=END,
        data_subdir=DATA_SUBDIR,
        features_subdir=FEATURES_SUBDIR,
        models_subdir=MODELS_SUBDIR,
        selection_file=SELECTION_FILE,
        enable_openvino_convert=ENABLE_OPENVINO_CONVERT,
        openvino_out_subdir=OPENVINO_OUT_SUBDIR,
    ),
)
print('Submitted run id:', run.run_id)
